In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from google.colab import files
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0


Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting

In [0]:
from google.colab import auth
auth.authenticate_user()

In [4]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [5]:
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in Drive:')
!ls drive/Machine_Learning/


Files in Drive:
model.json  RNN Chatbot.ipynb  train  wonderland.txt


In [0]:
filename = "drive/Machine_Learning/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [0]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [8]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  163779
Total Vocab:  58


In [0]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

In [10]:
print ("Total Patterns: ", n_patterns)

Total Patterns:  163679


In [0]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

In [0]:
X = X / float(n_vocab)

In [0]:
y = np_utils.to_categorical(dataY)

In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.fit(X, y, epochs=80, batch_size=256)

Epoch 1/80
163679/163679 [==============================] - 162s 988us/step - loss: 3.0307 - acc: 0.1752
Epoch 2/80
 47104/163679 [=======>......................] - ETA: 1:54 - loss: 2.9017 - acc: 0.2017

163679/163679 [==============================] - 160s 980us/step - loss: 2.8573 - acc: 0.2123
Epoch 3/80
 80384/163679 [=============>................] - ETA: 1:21 - loss: 2.7858 - acc: 0.2300

163679/163679 [==============================] - 160s 980us/step - loss: 2.7691 - acc: 0.2341
Epoch 4/80
 95488/163679 [================>.............] - ETA: 1:07 - loss: 2.7259 - acc: 0.2425

163679/163679 [==============================] - 162s 987us/step - loss: 2.7157 - acc: 0.2445
Epoch 5/80
102400/163679 [=================>............] - ETA: 1:00 - loss: 2.6686 - acc: 0.2546

163679/163679 [==============================] - 161s 983us/step - loss: 2.6612 - acc: 0.2555
Epoch 6/80
105472/163679 [==================>...........] - ETA: 57s - loss: 2.6123 - acc: 0.2664

...

163679/163679 [==============================] - 161s 981us/step - loss: 1.6227 - acc: 0.5147
Epoch 77/80
107008/163679 [==================>...........] - ETA: 55s - loss: 1.6185 - acc: 0.5160

163679/163679 [==============================] - 161s 984us/step - loss: 1.6452 - acc: 0.5092
Epoch 78/80
107008/163679 [==================>...........] - ETA: 55s - loss: 1.6180 - acc: 0.5161

163679/163679 [==============================] - 161s 981us/step - loss: 1.6246 - acc: 0.5142
Epoch 79/80
107008/163679 [==================>...........] - ETA: 55s - loss: 1.6009 - acc: 0.5206

163679/163679 [==============================] - 160s 979us/step - loss: 1.6097 - acc: 0.5179
Epoch 80/80
107008/163679 [==================>...........] - ETA: 55s - loss: 1.5953 - acc: 0.5220

163679/163679 [==============================] - 161s 982us/step - loss: 1.6044 - acc: 0.5192


In [16]:
model_json=model.to_json()
with open("drive/Machine_Learning/model.json", "w") as json_file:
  json_file.write(model_json)
model.save_weights("drive/Machine_Learning/model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [18]:
json_file = open('drive/Machine_Learning/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("model.h5")
print("Loaded model from disk")

OSError: ignored

In [0]:
start = numpy.random.randint(0, len(dataX)-1)
sentence="d herself lying on the bank, with her head in the lap of her sister, who was gently brushing away so"
sentence=sentence.lower()
x1=[char_to_int[value] for value in sentence]
pattern = x1
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
print("-------")
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")